# **Few Shot Learning**
is a machine learning framework in which an AI model learns to make accurate predictions by training on a very small number of labeled examples

In [1]:
!pip install -q langchain==0.0.219 openai==0.27.8

# !pip install -q transformers==4.29.2 sentencepiece==0.1.99 accelerate==0.19.0 bitsandbytes==0.39.0

!pip install -q python-dotenv==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00


In [2]:
from dotenv import dotenv_values
env_values = dotenv_values("./app.env")

openai_api_key = env_values['OPEN_API_KEY']

In [3]:
from langchain.llms import OpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [25]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.5, openai_api_key=openai_api_key)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [26]:
demo_message = """
Country Name: Egypt | Capital Name: Cairo
Country Name: Saudi Arabia | Capital Name: Riyadh
Country Name: France | Capital Name: Paris
Country Name: China |
""".strip()

print( llm( demo_message ) )

Capital Name: Beijing


In [27]:
demo_template = PromptTemplate(
    template="Country Name: {country} | Capital Name: {capital}",
    input_variables=['country', 'capital']
)

examples = [
    {"country": "Egypt", "capital": "Cairo"},
    {"country": "Saudi Arabia", "capital": "Riyadh"},
    {"country": "France", "capital": "Paris"},
]

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=demo_template,
    suffix="Country Name: {country} |",
    input_variables=['country']
)

In [28]:
print( llm( prompt.format(country='Germany') ) )

 Capital Name: Berlin


# Riddles

In [29]:
riddle_1 = """
I have five oranges, and the sum of what my sister and brother have is three times what I have plus thirty-five
""".strip()

print( llm( riddle_1 ) )

Let's represent what you have as x.

You have 5 oranges, so x = 5.

The sum of what your sister and brother have is three times what you have plus thirty-five, so the total number of oranges they have is 3x + 35.

Let's substitute x = 5 into the equation:

3(5) + 35 = 15 + 35 = 50

Therefore, your sister and brother have a total of 50 oranges.


In [30]:
riddle_3 = """
When I was seven, my sister was twice my age. Now I am seventy years old, how old can my sister be?
""".strip()

print( llm( riddle_3 ) )

If your sister was twice your age when you were seven, that means she was 14 at that time. Since then, 63 years have passed (70 - 7 = 63), so your sister would be 77 years old now.


In [ ]:
examples = [
    {
        "question": "When I was seven, my sister was twice my age. Now I am seventy years old, how old can my sister be?",
        "answer": "\n".join([
            "We will followup some questions to get the answer.",
            "Follow up: How old was your sister when you were seven?",
            "Intermediate answer: Twice, which mean 14 years.",
            "Follow up: What is the difference between your age and your sister's age?",
            "Intermediate answer: 14 years - 7 years = 7 years.",
            "Follow up: When you were seventy years old, how old would your sister be?",
            "Intermediate answer: my age (70) +  The difference between me and my sister's age (7) = 77 years.",
            "Final Answer: 77 years."
        ])
    },
    {
        "question": "I have five oranges, and the sum of what my sister and brother have is three times what I have plus thirty-five?",
        "answer": "\n".join([
            "We will followup some questions to get the answer.",
            "Follow up: How many oranges do you have?",
            "Intermediate answer: 5 oranges.",
            "Follow up: How many oranges does your sister and brother have?",
            "Intermediate answer: three times: = 3 * 5 = 15. Also, we need to add thirty-five to them: 15 + 35 = 50 ",
            "Final Answer: 50 oranges."
        ])
    }
]

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

message = prompt.format(input="Total with what my family is 50 oranges. If we subtract one-fifth of the number from them, and add ten more oranges, how many oranges will there be in the end?")
print( llm(message) )

## AS we could see after update of openAI model the gpt 3.5  could solve equation and solve some tricks like human so no need in  fewshotprompt in this but we add some example of how we do it